In [1]:
from configuracoes_notebooks import set_proj_dir
set_proj_dir()

O diretorio do seu projeto é coleta_cebrap
Caminho absoluto do diretorio encontrado C:\Users\Olívia\Desktop\batatinhas\coleta_cebrap
Caminho no path.


In [2]:
import geopandas as gpd
import os

from notebooks.jupyter import utils
from utils import (
    get_data_diretorio,
    save_parquet_excel,
)

# Ocorrências de inundações por distrito
Com base nas intersecções que temos, podemos somar os pontas para sabermos o total em cada distrito

In [3]:
data_path = get_data_diretorio()
assets_path = os.path.join(
    data_path,
    'assets'
)

# Dependências
Este notebook é dependente dos arquivos resultantes dos notebooks "overlay_pontos_inundacao_2024" e "../../arborizacao_viaria/malha_distritos"

In [4]:
gdf_distritos = gpd.read_parquet(
    os.path.join(
        assets_path,
        'distrito_ibge.parquet'
    )
)

In [5]:
overlay_inund = gpd.read_parquet(
    os.path.join(
        assets_path,
        'areas_inundacao',
        'interseccao_inundacoes2024_distritos.parquet'
    )
)

# Soma de pontos por distrito

In [6]:
for i, distrito in enumerate(gdf_distritos['CD_DIST']):
    gdf_distritos.loc[gdf_distritos['CD_DIST']==distrito, 'qt_inund_ocor'] = (
        len(
            overlay_inund
            .loc[overlay_inund['CD_DIST'] == distrito]
        )
    )

In [7]:
gdf_distritos.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_inund_ocor
53,3550308,São Paulo,355030854,MORUMBI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.462372,43690,20710,"POLYGON ((325192.046 7387331.143, 325166.217 7...",0.0
54,3550308,São Paulo,355030855,PARELHEIROS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,151.459856,153687,62104,"POLYGON ((324472.495 7358116.496, 324448.399 7...",10.0


# Conferir

Distritos sem pontos de ocorrências de inundações:

In [8]:
zero_dists=(
    gdf_distritos
    .loc[gdf_distritos['qt_inund_ocor']==float(0)]
)
zero_dists.shape

(38, 15)

In [9]:
zero_dists.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_inund_ocor
94,3550308,São Paulo,355030895,SAO DOMINGOS,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,9.914590,88884,37746,"POLYGON ((321653.073 7399213.8, 321651.611 739...",0.0
53,3550308,São Paulo,355030854,MORUMBI,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.462372,43690,20710,"POLYGON ((325192.046 7387331.143, 325166.217 7...",0.0


Distrito com o maior número de ocorrências de inundações:

In [10]:
(
    gdf_distritos
    .loc[gdf_distritos['qt_inund_ocor']==gdf_distritos['qt_inund_ocor'].max()]
)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_inund_ocor
36,3550308,São Paulo,355030837,ITAQUERA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,14.699172,210960,85467,"POLYGON ((351668.035 7394259.261, 351646.256 7...",18.0


Distrito com o menor número de inundações (excluindo os sem ocorrência nenhuma):

In [11]:
min_positiv=(
    gdf_distritos
    .loc[gdf_distritos['qt_inund_ocor']>0]
)
min_dists=(
    gdf_distritos
    .loc[gdf_distritos['qt_inund_ocor']==min_positiv['qt_inund_ocor'].min()]
)
min_dists.shape

(23, 15)

In [12]:
min_dists.sample(2)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_inund_ocor
3,3550308,São Paulo,355030804,ARICANDUVA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,6.847188,89574,37225,"POLYGON ((345121.729 7390756.39, 345114.818 73...",1.0
6,3550308,São Paulo,355030807,BELA VISTA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,2.760756,60024,41033,"POLYGON ((331713.373 7392672.088, 331705.437 7...",1.0


Nós vimos que a subprefeitura com mais pontos de ocorrências de inundação é a de São Miguel Paulista. Vamos confirmar se mantém quando calculamos pelos distritos:

In [13]:
overlay_inund.columns

Index(['data', 'ocorrencia', 'nm_subpref', 'id_ocorrencias', 'CD_MUN',
       'NM_MUN', 'CD_DIST', 'NM_DIST', 'CD_RGINT', 'NM_RGINT', 'CD_RGI',
       'NM_RGI', 'CD_CONCURB', 'NM_CONCURB', 'AREA_KM2', 'total_pop',
       'total_dom', 'geometry'],
      dtype='object')

In [14]:
subpref_confere = gdf_distritos.copy()
for i, distrito in enumerate(subpref_confere['CD_DIST']):
    if overlay_inund['CD_DIST'].str.contains(distrito).any():
        subpref_confere.loc[subpref_confere['CD_DIST']==distrito, 'subpref']=(
            overlay_inund
            .loc[overlay_inund['CD_DIST']==distrito]
            .head(1)['nm_subpref']
            .values[0]
        )

In [15]:
subpref_confere.sample(5)

,CD_MUN,NM_MUN,CD_DIST,NM_DIST,CD_RGINT,NM_RGINT,CD_RGI,NM_RGI,CD_CONCURB,NM_CONCURB,AREA_KM2,total_pop,total_dom,geometry,qt_inund_ocor,subpref
87,3550308,São Paulo,355030888,VILA LEOPOLDINA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,6.988733,46875,20673,"POLYGON ((324497.89 7396536.095, 324494.684 73...",1.0,LA - LAPA
64,3550308,São Paulo,355030865,RAPOSO TAVARES,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.164313,117738,49006,"POLYGON ((319022.919 7388900.499, 318990.285 7...",5.0,BT - BUTANTA
33,3550308,São Paulo,355030834,IPIRANGA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,11.026457,116271,53804,"POLYGON ((338824.283 7388385.211, 338823.81 73...",0.0,NaN
38,3550308,São Paulo,355030839,JACANA,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,7.473932,87329,34705,"POLYGON ((339057.719 7403206.175, 339043.253 7...",1.0,JT - JACANA/TREMEMBE
15,3550308,São Paulo,355030816,CAMPO GRANDE,3501,São Paulo,350001,São Paulo,3550308,São Paulo/SP,12.991325,115925,50009,"POLYGON ((329675.249 7379393.452, 329715.641 7...",0.0,NaN


# Salvar GDF

In [16]:
save_parquet_excel(
    gdf_distritos,
    fname='qt_ocorrencias_inund_distrito',
    data_path = assets_path,
    data_subpath= 'areas_inundacao'
)